In [1]:
# Import Standard Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import math
import os
import random

# Import PyTorch Libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# Seed for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [2]:
# Read both dataset
with open('data/shakespeare_train.txt', 'r', encoding='utf8') as f:
    train_text = f.read()
with open('data/shakespeare_valid.txt', 'r', encoding='utf8') as f:
    val_text = f.read()

# Combine both of them
combined_text = train_text + val_text
vocab = sorted(list(set(combined_text)))

# Mapping (like the instruction)
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))

# Vocab size
vocab_size = len(vocab_to_int)

# Define Dataset Class
# Reference: https://docs.pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html
class ShakespeareDataset(Dataset):
    def __init__(self, filepath, sequence_length, vocab_to_int_map, int_to_vocab_map):
        self.sequence_length = sequence_length
        self.vocab_to_int = vocab_to_int_map
        self.int_to_vocab = int_to_vocab_map
        self.vocab_size = len(vocab_to_int_map)
        
        # Open the txt
        with open(filepath, 'r', encoding='utf8') as f:
            text = f.read()
        
        # Encode (like in the instruction)
        self.data = np.array([self.vocab_to_int[c] for c in text], dtype=np.int64)

    def __len__(self):
        return len(self.data) - self.sequence_length

    # Reference: https://www.kaggle.com/datasets/creatednull/dataset
    def __getitem__(self, idx):
        # Define the start and end index
        start_index = idx
        end_index = idx + self.sequence_length + 1
        # Slice the data
        chunk = self.data[start_index:end_index]
        # Explicitly cast to torch.long for indices/labels
        input_seq = torch.tensor(chunk[:-1], dtype=torch.long)
        target_seq = torch.tensor(chunk[1:], dtype=torch.long)
        return input_seq, target_seq
    
    def get_vocab_size(self):
        return self.vocab_size

In [3]:
# Define the RNN/LSTM Model
# Reference: https://github.com/spro/char-rnn.pytorch/blob/master/model.py
class TextModel(nn.Module):
    def __init__(self, vocab_size, hidden_size, num_layers=1, model_type='RNN'):
        super(TextModel, self).__init__()
        # Initialize some variables
        self.hidden_dim = hidden_size
        self.num_layers = num_layers
        self.model_type = model_type
        
        # Embedding layer for one hot encoding (required based on the instruction)
        self.embedding = nn.Embedding(vocab_size, self.hidden_dim)
        
        # RNN and LSTM layers
        if model_type == 'RNN':
            self.rnn = nn.RNN(input_size=self.hidden_dim, hidden_size=self.hidden_dim, num_layers=self.num_layers, batch_first=True)
        elif model_type == 'LSTM':
            self.rnn = nn.LSTM(input_size=self.hidden_dim, hidden_size=self.hidden_dim, num_layers=self.num_layers, batch_first=True)
        else:
            raise ValueError("model_type must be either 'RNN' or 'LSTM'")
        
        # Fully connected (output) layer
        self.output_layer = nn.Linear(self.hidden_dim, vocab_size)
    
    # Reference: https://colab.research.google.com/github/agungsantoso/deep-learning-v2-pytorch/blob/master/sentiment-rnn/Sentiment_RNN_Exercise.ipynb
    def forward(self, x, hidden):        
        # Embedding and out (either from RNN or LSTM)
        embeds = self.embedding(x)
        out, hidden = self.rnn(embeds, hidden)
        # Stack up the model's outputs
        out = out.contiguous().view(-1, self.hidden_dim)
        # Pass to the fully connected layer
        out = self.output_layer(out)
        return out, hidden
    
    # Reference: https://stackoverflow.com/questions/53914450/problems-with-lstm-model
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        if self.model_type == 'RNN':
            return weight.new(self.num_layers, batch_size, self.hidden_dim).zero_()
        elif self.model_type == 'LSTM':
            return (weight.new(self.num_layers, batch_size, self.hidden_dim).zero_(), weight.new(self.num_layers, batch_size, self.hidden_dim).zero_())

# Print the architecture of the RNN
print("RNN Architecture: ")
rnn_architecture = TextModel(vocab_size, hidden_size=128, num_layers=2, model_type='RNN').to(device)
print(rnn_architecture)

print()

# Print the architecture of the LSTM
print("LSTM Architecture: ")
lstm_architecture = TextModel(vocab_size, hidden_size=128, num_layers=2, model_type='LSTM').to(device)
print(lstm_architecture)

RNN Architecture: 
TextModel(
  (embedding): Embedding(67, 128)
  (rnn): RNN(128, 128, num_layers=2, batch_first=True)
  (output_layer): Linear(in_features=128, out_features=67, bias=True)
)

LSTM Architecture: 
TextModel(
  (embedding): Embedding(67, 128)
  (rnn): LSTM(128, 128, num_layers=2, batch_first=True)
  (output_layer): Linear(in_features=128, out_features=67, bias=True)
)


In [ ]:
# bits-per-character (BPC) based on the instruction
# Reference: https://stats.stackexchange.com/questions/211858/how-to-compute-bits-per-character-bpc
def bpc_loss(outputs, targets):
    # Calculate cross-entropy loss in nats
    cross_entropy_loss = F.cross_entropy(outputs, targets.view(-1))
    # Calculate the BPC
    bpc = cross_entropy_loss / math.log(2.0)
    return bpc

# Calculate the error rate
# Reference: https://docs.pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
def calculate_error_rate(outputs, targets):
    targets_flat = targets.view(-1)
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == targets_flat).sum().item()
    total = targets_flat.size(0)
    error_rate = 1.0 - (correct / total)
    return error_rate

# Reference: https://pytorch.org/tutorials/intermediate/char_rnn_generation_tutorial.html
def generate_text(model, dataset, start_str="JULIET:", predict_len=500, temperature=0.8, device='cpu'):
    model.eval()
    # Fresh hidden state
    train_hidden = model.init_hidden(batch_size=1)
    
    # Convert start_str to integers
    try:
        vocabs = []
        for s in start_str:
            vocabs.append(dataset.vocab_to_int[s])
    except KeyError as e:
        print(f"Error: Character '{e.args[0]}' in start_str is not in the vocabulary.")
        return
    
    # Tensor for priming sequence
    warmup_tensor = torch.tensor([vocabs], dtype=torch.long).to(device)
    _, train_hidden = model(warmup_tensor, train_hidden)
    next_char_tensor = torch.tensor([[vocabs[-1]]], dtype=torch.long).to(device)
    predicted_text = start_str
    
    with torch.no_grad():
        for _ in range(predict_len):
            output, train_hidden = model(next_char_tensor, train_hidden)
            # Inspired by: https://gist.github.com/karpathy/d4dee566867f8291f086
            output_dist = output.data.view(-1).div(temperature).exp()

            predicted_index = torch.multinomial(output_dist, 1)[0]

            predicted_char = dataset.int_to_vocab[predicted_index.item()]
            predicted_text += predicted_char
            
            next_char_tensor = torch.tensor([[predicted_index.item()]], dtype=torch.long).to(device)
            
    model.train()
    return predicted_text

# Reference: https://github.com/pytorch/examples/tree/main/word_language_model
def train_val_cycle(model, train_loader, val_loader, train_dataset, num_epochs, lr, device):
    # Use Adam optimizer
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    # Logging for plots
    log = {
        'train_loss': [],
        'val_loss': [],
        'train_error': [],
        'val_error': []
    }
    
    step_count = 0
    # Breakpoint logic 
    total_steps = len(train_loader) * num_epochs
    breakpoint_interval = total_steps // 5
    
    # Training loop
    for epoch in range(num_epochs):
        # Training state
        model.train()
        train_hidden = model.init_hidden(train_loader.batch_size)
        
        train_loss_sum = 0
        train_error_sum = 0

        # Use tqdm for training loop
        train_tqdm = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]")
        
        # Training loop
        for inputs, targets in train_tqdm:
            inputs, targets = inputs.to(device), targets.to(device)
            
            # Reference: https://discuss.pytorch.org/t/solved-why-we-need-to-detach-variable-which-contains-hidden-representation/1426
            if model.model_type == 'RNN':
                train_hidden = train_hidden.detach()
            # Reference: https://lightning.ai/lightning-ai/studios/train-a-recurrent-neural-network-with-pytorch-lightning
            elif model.model_type == 'LSTM':
                train_hidden = (train_hidden[0].detach(), train_hidden[1].detach())
            
            # Forward pass
            optimizer.zero_grad()
            output, train_hidden = model(inputs, train_hidden)
            
            # Calculate metrics
            train_loss = bpc_loss(output, targets)
            train_error = calculate_error_rate(output, targets)
            
            # Backward pass
            train_loss.backward()
            # Reference: https://pytorch.org/docs/stable/generated/torch.nn.utils.clip_grad_norm_.html
            nn.utils.clip_grad_norm_(model.parameters(), 5)
            optimizer.step()
            
            train_loss_sum += train_loss.item()
            train_error_sum += train_error
            step_count += 1
            
            # Printing the breakpoint text
            if step_count % breakpoint_interval == 0:
                print(f"Progress: {step_count}/{total_steps} steps done ({step_count/total_steps * 100:.0f}%)")
                generated_text = generate_text(model, train_dataset, start_str="JULIET:", predict_len=200, device=device)
                print(generated_text)

        # Validation state
        model.eval()
        val_loss_sum = 0
        val_error_sum = 0
        val_hidden = model.init_hidden(val_loader.batch_size)
        
        # Use tqdm for validation loop
        val_tqdm = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]")
        
        with torch.no_grad():
            # Validation loop
            for inputs, targets in val_tqdm:
                inputs, targets = inputs.to(device), targets.to(device)
                
                output, val_hidden = model(inputs, val_hidden)
                
                val_loss = bpc_loss(output, targets)
                val_error = calculate_error_rate(output, targets)
                
                val_loss_sum += val_loss.item()
                val_error_sum += val_error

        # Log for loss and error
        avg_train_loss = train_loss_sum / len(train_loader)
        avg_train_error = train_error_sum / len(train_loader)
        avg_val_loss = val_loss_sum / len(val_loader)
        avg_val_error = val_error_sum / len(val_loader)
        
        log['train_loss'].append(avg_train_loss)
        log['train_error'].append(avg_train_error)
        log['val_loss'].append(avg_val_loss)
        log['val_error'].append(avg_val_error)
        
        print(f"Epoch {epoch+1}: Train Loss (BPC): {avg_train_loss:.4f}, Train Error: {avg_train_error:.4f}, "
              f"Val Loss (BPC): {avg_val_loss:.4f}, Val Error: {avg_val_error:.4f}\n")

    return log

# Reference: https://matplotlib.org/stable/tutorials/introductory/pyplot.html
def plot_loss_error(log, model_type, hidden_size, seq_len, epochs, plot_dir="figures"):
    # Create directory if not exists
    if not os.path.exists(plot_dir):
        os.makedirs(plot_dir)

    # So the epoch start from 1
    epochs_range = range(1, epochs + 1)
    
    # Loss plots
    for key, label, color in [('train_loss', 'Training Loss (BPC)', 'b'), ('val_loss', 'Validation Loss (BPC)', 'r')]:
        plt.figure(figsize=(8,6))
        plt.plot(epochs_range, log[key], f'{color}-o', label=label)
        plt.title(f'{label} [{model_type}, HS={hidden_size}, SL={seq_len}]')
        plt.xlabel('Epochs')
        plt.ylabel(label)
        plt.legend()
        plt.grid(True)
        plt.savefig(f"{plot_dir}/{key}_{model_type}_HS{hidden_size}_SL{seq_len}.png")
        plt.close()

    # Error rate plots
    for key, label, color in [('train_error', 'Training Error', 'g'), ('val_error', 'Validation Error', 'm')]:
        plt.figure(figsize=(8,6))
        plt.plot(epochs_range, log[key], f'{color}-o', label=label)
        plt.title(f'{label} [{model_type}, HS={hidden_size}, SL={seq_len}]')
        plt.xlabel('Epochs')
        plt.ylabel('Error Rate')
        plt.legend()
        plt.grid(True)
        plt.savefig(f"{plot_dir}/{key}_{model_type}_HS{hidden_size}_SL{seq_len}.png")
        plt.close()

# Reference: https://matplotlib.org/stable/tutorials/introductory/pyplot.html
def plot_comparison_1(results_logs, hidden_sizes, seq_lengths, epochs, plot_dir="figures"):
    if not os.path.exists(plot_dir):
        os.makedirs(plot_dir)
        
    epochs_range = range(1, epochs + 1)
    models = ['RNN', 'LSTM']
    # First comparison: hidden sizes
    for model_type in models:
        # Training loss
        plt.figure(figsize=(12, 6))
        for h in hidden_sizes:
            log = results_logs[(model_type, 'hidden', h)]
            plt.plot(epochs_range, log['train_loss'], label=f'HS={h}')
        plt.title(f'{model_type} Training Loss vs Epoch (by Hidden Size)')
        plt.xlabel('Epochs')
        plt.ylabel('Training Loss (BPC)')
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(f"{plot_dir}/{model_type}_train_loss_vs_hs.png")
        plt.close()

        # Validation loss
        plt.figure(figsize=(12, 6))
        for h in hidden_sizes:
            log = results_logs[(model_type, 'hidden', h)]
            plt.plot(epochs_range, log['val_loss'], label=f'HS={h}')
        plt.title(f'{model_type} Validation Loss vs Epoch (by Hidden Size)')
        plt.xlabel('Epochs')
        plt.ylabel('Validation Loss (BPC)')
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(f"{plot_dir}/{model_type}_val_loss_vs_hs.png")
        plt.close()

    # Second Comparison: sequence lengths
    for model_type in models:
        # Training loss
        plt.figure(figsize=(12, 6))
        for s in seq_lengths:
            log = results_logs[(model_type, 'seq_len', s)]
            plt.plot(epochs_range, log['train_loss'], label=f'SL={s}')
        plt.title(f'{model_type} Training Loss vs Epoch (by Sequence Length)')
        plt.xlabel('Epochs')
        plt.ylabel('Training Loss (BPC)')
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(f"{plot_dir}/{model_type}_train_loss_vs_sl.png")
        plt.close()

        # Validation loss
        plt.figure(figsize=(12, 6))
        for s in seq_lengths:
            log = results_logs[(model_type, 'seq_len', s)]
            plt.plot(epochs_range, log['val_loss'], label=f'SL={s}')
        plt.title(f'{model_type} Validation Loss vs Epoch (by Sequence Length)')
        plt.xlabel('Epochs')
        plt.ylabel('Validation Loss (BPC)')
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(f"{plot_dir}/{model_type}_val_loss_vs_sl.png")
        plt.close()

# Reference 1: https://stackoverflow.com/questions/53914450/problems-with-lstm-model
# Reference 2: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.xticks.html
def plot_comparison_2(results_log, hidden_sizes, seq_lengths, plot_dir="figures"):
    # Retrieve the data from the log for Hidden State Size (Training)
    rnn_losses = [results_log[('RNN', 'hidden', h)]['train_loss'][-1] for h in hidden_sizes]
    lstm_losses = [results_log[('LSTM', 'hidden', h)]['train_loss'][-1] for h in hidden_sizes]
    
    # Plot Loss vs Hidden State Size
    plt.figure(figsize=(8, 6))
    plt.plot(hidden_sizes, rnn_losses, 'b-o', label='RNN')
    plt.plot(hidden_sizes, lstm_losses, 'r-o', label='LSTM')
    plt.title('Final Training Loss vs. Hidden State Size (Fixed SeqLen=100)')
    plt.xlabel('Hidden State Size')
    plt.ylabel('Final Training Loss (BPC)')
    plt.xticks(hidden_sizes)
    plt.legend()
    plt.grid(True)
    plt.savefig(f"{plot_dir}/train_loss_vs_hidden.png")
    plt.close()

    # Retrieve the data from the log for Hidden State Size
    rnn_losses = [results_log[('RNN', 'hidden', h)]['val_loss'][-1] for h in hidden_sizes]
    lstm_losses = [results_log[('LSTM', 'hidden', h)]['val_loss'][-1] for h in hidden_sizes]
    
    # Plot Loss vs Hidden State Size
    plt.figure(figsize=(8, 6))
    plt.plot(hidden_sizes, rnn_losses, 'b-o', label='RNN')
    plt.plot(hidden_sizes, lstm_losses, 'r-o', label='LSTM')
    plt.title('Final Validation Loss vs. Hidden State Size (Fixed SeqLen=100)')
    plt.xlabel('Hidden State Size')
    plt.ylabel('Final Validation Loss (BPC)')
    plt.xticks(hidden_sizes)
    plt.legend()
    plt.grid(True)
    plt.savefig(f"{plot_dir}/val_loss_vs_hidden.png")
    plt.close()

    # Retrieve the data from the log for Sequence Length
    rnn_losses = [results_log[('RNN', 'seq_len', s)]['train_loss'][-1] for s in seq_lengths]
    lstm_losses = [results_log[('LSTM', 'seq_len', s)]['train_loss'][-1] for s in seq_lengths]
        
    # Plot Loss vs Sequence Length
    plt.figure(figsize=(8, 6))
    plt.plot(seq_lengths, rnn_losses, 'b-o', label='RNN')
    plt.plot(seq_lengths, lstm_losses, 'r-o', label='LSTM')
    plt.title('Final Training Loss vs. Sequence Length (Fixed HiddenSize=128)')
    plt.xlabel('Sequence Length')
    plt.ylabel('Final Training Loss (BPC)')
    plt.xticks(seq_lengths)
    plt.legend()
    plt.grid(True)
    plt.savefig(f"{plot_dir}/train_loss_vs_seqlen.png")
    plt.close()

    # Retrieve the data from the log for Sequence Length
    rnn_losses = [results_log[('RNN', 'seq_len', s)]['val_loss'][-1] for s in seq_lengths]
    lstm_losses = [results_log[('LSTM', 'seq_len', s)]['val_loss'][-1] for s in seq_lengths]
        
    # Plot Loss vs Sequence Length
    plt.figure(figsize=(8, 6))
    plt.plot(seq_lengths, rnn_losses, 'b-o', label='RNN')
    plt.plot(seq_lengths, lstm_losses, 'r-o', label='LSTM')
    plt.title('Final Validation Loss vs. Sequence Length (Fixed HiddenSize=128)')
    plt.xlabel('Sequence Length')
    plt.ylabel('Final Validation Loss (BPC)')
    plt.xticks(seq_lengths)
    plt.legend()
    plt.grid(True)
    plt.savefig(f"{plot_dir}/val_loss_vs_seqlen.png")
    plt.close()

In [ ]:
# Hyperparameters for experiments
hidden_sizes = [64, 128, 256]
seq_lengths = [50, 100, 200]
model_types = ['RNN', 'LSTM']

# Fixed hyperparameters
epochs = 15
lr = 0.002
batch_size = 512
fixed_seq_len = 100
fixed_hidden_size = 128

# For plotting later
results_log = {}

# Load the dataset for hidden size experiment
train_dataset_hs = ShakespeareDataset('data/shakespeare_train.txt', fixed_seq_len, vocab_to_int, int_to_vocab)
val_dataset_hs = ShakespeareDataset('data/shakespeare_valid.txt', fixed_seq_len, vocab_to_int, int_to_vocab)

# Dataloaders for hidden size experiment
train_loader_hs = DataLoader(train_dataset_hs, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=8, pin_memory=True)
val_loader_hs = DataLoader(val_dataset_hs, batch_size=batch_size, shuffle=False, drop_last=True, num_workers=8, pin_memory=True)

# Experiment 1: Vary Hidden Size (Fix Sequence Length)
for model_type in model_types:
    for hs in hidden_sizes:
        print(f"\nModel Type: {model_type}, Hidden Size={hs}, Sequence Length={fixed_seq_len}")
        # Call the model
        model = TextModel(vocab_size, hs, num_layers=2, model_type=model_type).to(device)
        # Training and Validation
        log = train_val_cycle(model, train_loader_hs, val_loader_hs, train_dataset_hs, epochs, lr, device)
        
        # Plot learning curves and error rates
        plot_loss_error(log, model_type, hs, fixed_seq_len, epochs, plot_dir="figures")
        # Store the complete log for comparison plots later
        # Reference: https://stackoverflow.com/questions/1938614/in-what-case-would-i-use-a-tuple-as-a-dictionary-key
        results_log[(model_type, 'hidden', hs)] = log

# Add the seq_len=100 to the log, so we can skip it
results_log[('RNN', 'seq_len', fixed_seq_len)] = results_log[('RNN', 'hidden', fixed_hidden_size)]
results_log[('LSTM', 'seq_len', fixed_seq_len)] = results_log[('LSTM', 'hidden', fixed_hidden_size)]

# Skipping sequence length 100 as it is already done
final_seq_lengths = [50, 200]

# Experiment 2: Vary Sequence Length (Fix Hidden Size)
for model_type in model_types:
    for sl in final_seq_lengths:
        print(f"\nModel Type: {model_type}, Hidden Size={fixed_hidden_size}, Sequence Length={sl}")
        
        # Dataset with new sequence length
        train_dataset_sl = ShakespeareDataset('data/shakespeare_train.txt', sl, vocab_to_int, int_to_vocab)
        val_dataset_sl = ShakespeareDataset('data/shakespeare_valid.txt', sl, vocab_to_int, int_to_vocab)

        # Dataloaders for sequence length experiment
        train_loader_sl = DataLoader(train_dataset_sl, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=8, pin_memory=True)
        val_loader_sl = DataLoader(val_dataset_sl, batch_size=batch_size, shuffle=False, drop_last=True, num_workers=8, pin_memory=True)
        
        # Call the model
        model = TextModel(vocab_size, fixed_hidden_size, num_layers=2, model_type=model_type).to(device)
        # Training and Validation
        log = train_val_cycle(model, train_loader_sl, val_loader_sl, train_dataset_sl, epochs, lr, device)
        
        # Plot learning curves and error rates
        plot_loss_error(log, model_type, fixed_hidden_size, sl, epochs, plot_dir="figures")
        # Store the complete log for comparison plots later
        # Reference: https://stackoverflow.com/questions/1938614/in-what-case-would-i-use-a-tuple-as-a-dictionary-key
        results_log[(model_type, 'seq_len', sl)] = log

plot_comparison_1(results_log, hidden_sizes, seq_lengths, epochs, plot_dir="figures")
plot_comparison_2(results_log, hidden_sizes, seq_lengths, plot_dir="figures")


Model Type: RNN, Hidden Size=64, Sequence Length=100


Epoch 1/15 [Val]: 100%|██████████| 433/433 [00:00<00:00, 1086.27it/s]


Epoch 1: Train Loss (BPC): 2.4023, Train Error: 0.4901, Val Loss (BPC): 2.4812, Val Error: 0.4953



Epoch 2/15 [Val]: 100%|██████████| 433/433 [00:00<00:00, 1094.95it/s]


Epoch 2: Train Loss (BPC): 2.3011, Train Error: 0.4723, Val Loss (BPC): 2.4835, Val Error: 0.4935



...

Progress: 127470/127470 steps done (100%)


Epoch 15/15 [Train]: 100%|██████████| 8498/8498 [02:55<00:00, 48.29it/s]


JULIET:
Rugilled thee.
Then the reward under my be the truth,
The best of my husband from our despised feats;
And so I doubt not by the blessed banks,
That no present dreeues itself still: discuss,
That evil


Epoch 15/15 [Val]: 100%|██████████| 433/433 [00:03<00:00, 115.88it/s]


Epoch 15: Train Loss (BPC): 1.7495, Train Error: 0.3766, Val Loss (BPC): 2.1639, Val Error: 0.4318



In [6]:
# Generate text using the last model
final_text = generate_text(model, train_dataset_sl, start_str="JULIET:", predict_len=1500, temperature=0.7, device=device)
print(final_text)

# Save the generated text to a txt file
with open("figures/final_generation.txt", "w", encoding="utf-8") as f:
    f.write(final_text)

JULIET:
Good lad is slain; and my husband you be
As you see my language than a son.

ANTONIO:
Good 'ome the jest of any of your trick.

COMINIUS:
You, Pistol, I warranch him the indiricious instruction or I am in love!

Messenger:
My lord, 'tis true: if I love me to see the eldest man
piece of my Lord Corily and her jewel well.

KING EDWARD IV:
I say, there is a man. He call'd the gentleman:
O, give them stand upon the rest and last in the
presence of your money.

MARK ANTONY:
Words with me to seek him out of it.

HAMLET:
The reverend king of me be a charge to a sin.

PETRUCHIO:
Cates and keeping monster's content! I have decieth
In seven Cressid me a woman, for one present
clock, though 'tis my love.

DUNCAN:
She hath churly slain.

CORIOLANUS:
I will have her well enough.

PROSPERO:
Henes, my lord, Alencon will I be.

THALIARD:
My lord, now this is his husband for you.

BARDOLPH:
Pray you, do you go to my point.

ROMEO:
And such a shadow shall be glad of this hindest spoke of the ben